In [1]:
import os, sys
import pandas as pd
import numpy as np
import pickle

In [2]:
patient_path = "patient_clean.csv"
patient_df = pd.read_csv(patient_path)

In [15]:
len(set(patient_df.patientunitstayid))

200859

In [5]:
micu = patient_df[patient_df.unittype == 'MICU']
micuAge = micu[micu.age >= 18]

In [6]:
readmit = micuAge.groupby('patienthealthsystemstayid')['patientunitstayid'].count()
readmit_labels = (readmit > 1).astype('int64').to_frame().rename(columns={'patientunitstayid':'readmission'})

In [7]:
firstIcus = micuAge.loc[micuAge.groupby('patienthealthsystemstayid').hospitaladmitoffset.idxmax()]

In [8]:
readmission_cohort = firstIcus.join(readmit_labels, on='patienthealthsystemstayid')

In [9]:
(readmission_cohort['unitdischargeoffset'] < 60*24).sum()

4150

In [10]:
cohort = readmission_cohort

In [11]:
cohort['mortality'] = (cohort['unitdischargestatus'] == 'Expired').astype('int64')

In [12]:
cohort['losday'] = (cohort['unitdischargeoffset'].astype('float') / (24.*60.))

In [13]:
cohort['los>3day'] = (cohort['losday'] > 3.).astype('int64')
cohort['los>7day'] = (cohort['losday'] > 7.).astype('int64')

In [14]:
cohort['readmission'].sum()

1739

In [15]:
cohort12h = cohort[cohort['unitdischargeoffset'] > 60*12]

In [16]:
cohort12h.shape

(13941, 34)

In [17]:
dxPath = 'diagnosis.csv'
dxDf = pd.read_csv(dxPath)

In [18]:
cohort12hDx = dxDf[dxDf.patientunitstayid.isin(cohort12h.patientunitstayid)]

In [19]:
diagnosis = cohort12hDx.groupby('patientunitstayid')['diagnosisstring'].apply(list).to_frame()

In [312]:
tempdf = cohort12h.join(diagnosis, on='patientunitstayid')

In [321]:
pickle.dump(tempdf, open('eicu_cohort.pk', 'wb'), -1)

In [29]:
cohort = pickle.load(open('eicu_cohort.pk', 'rb'))

In [30]:
cohort

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid,readmission,mortality,losday,los>3day,los>7day,diagnosisstring
32,141366,129086,Male,81.0,Caucasian,73,97,"Respiratory surgery, other",172.7,16:06:00,...,1339,Floor,Alive,002-17331,0,0,0.929861,0,0,NaN
34,141392,129109,Female,78.0,Caucasian,73,97,"Sepsis, renal/UTI (including bladder)",160.0,08:32:00,...,961,Floor,Alive,002-49989,0,0,0.667361,0,0,[endocrine|glucose metabolism|diabetes mellitu...
42,141462,129166,Male,80.0,Caucasian,73,97,"Sepsis, pulmonary",170.2,05:27:00,...,5198,Floor,Alive,002-24032,0,0,3.609722,1,0,[endocrine|glucose metabolism|diabetes mellitu...
44,141475,129176,Male,87.0,Caucasian,73,97,"Sepsis, pulmonary",180.3,06:48:00,...,732,Floor,Alive,002-11644,0,0,0.508333,0,0,NaN
55,141584,129260,Male,63.0,Caucasian,73,97,Emphysema/bronchitis,185.4,01:10:00,...,4225,Floor,Alive,002-17958,1,0,2.934028,0,0,"[endocrine|thyroid|hypothyroidism, pulmonary|d..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200843,3353190,2743049,Male,39.0,Caucasian,458,1106,Coma/change in level of consciousness (for hep...,180.3,19:35:00,...,3051,Floor,Alive,035-7770,0,0,2.118750,0,0,[hematology|coagulation disorders|hypercoagula...
200847,3353198,2743055,Female,66.0,Caucasian,458,1106,Coma/change in level of consciousness (for hep...,157.5,06:38:00,...,23031,Step-Down Unit (SDU),Alive,035-18808,0,0,15.993750,1,1,[neurologic|altered mental status / pain|chang...
200851,3353213,2743064,Female,51.0,Caucasian,458,1106,Coma/change in level of consciousness (for hep...,160.0,02:13:00,...,4913,Home,Alive,035-15653,0,0,3.411806,1,0,[pulmonary|disorders of the airways|acute COPD...
200855,3353237,2743086,Female,79.0,Caucasian,458,1106,"Embolus, pulmonary",162.6,01:45:00,...,1269,Step-Down Unit (SDU),Alive,035-751,0,0,0.881250,0,0,[pulmonary|disorders of vasculature|pulmonary ...


In [34]:
cohort['los>7day'].mean()

0.102359945484542

In [20]:
dxDict = dict(enumerate(cohort12hDx.groupby('diagnosisstring').count().index))

In [21]:
#pd.DataFrame(dxDict)
dxDict = dict([(v,k) for k,v in dxDict.items()])

In [36]:
dxSet = set([key.split('|')[0] for key in dxDict.keys()])
#dxSet = set([key.split('|')[0] + " # " + key.split('|')[1] for key in dxDict.keys()])

In [37]:
(dxSet)

{'burns/trauma',
 'cardiovascular',
 'endocrine',
 'gastrointestinal',
 'general',
 'genitourinary',
 'hematology',
 'infectious diseases',
 'musculoskeletal',
 'neurologic',
 'obstetrics/gynecology',
 'oncology',
 'pulmonary',
 'renal',
 'surgery',
 'toxicology',
 'transplant'}

In [ ]:
'Infectious and parasitic diseases',
'Neoplasms',
'Endocrine; nutritional; and metabolic diseases and immunity disorders',
'Diseases of the blood and blood-forming organs',
'Mental illness',
'Diseases of the nervous system and sense organs',
'Diseases of the circulatory system',
'Diseases of the respiratory system',
'Diseases of the digestive system',
'Diseases of the genitourinary system',
'Complications of pregnancy; childbirth; and the puerperium',
'Diseases of the skin and subcutaneous tissue',
'Diseases of the musculoskeletal system and connective tissue',
'Congenital anomalies',
'Certain conditions originating in the perinatal period',
'Injury and poisoning',
'Symptoms; signs; and ill-defined conditions and factors influencing health status',
'Residual codes; unclassified; all E codes',